# Hydroelastic Contact: Nonconvex Mesh
For instructions on how to run these tutorial notebooks, please see the [index](./index.ipynb).

If you are not familiar with Drake's hydroelastic contact, study [hydroelastic_contact_basics.ipynb](./hydroelastic_contact_basics.ipynb). You can also find more information in Hydroelastic Contact User Guide [here.](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)

## Introduction

This tutorial shows you how to set up simulations using compliant-hydroelastic nonconvex meshes. We'll use a simple example of a bell pepper dropped onto a bowl on a table top, with all three objects represented by compliant-hydroelastic meshes. Contact forces are calculated and visualized.

In [1]:
import os
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform
from pydrake.multibody.meshcat import ContactVisualizer, ContactVisualizerParams
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlant, MultibodyPlantConfig
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import ApplyVisualizationConfig, ModelVisualizer, VisualizationConfig

## Start MeshCat

See the section [Viewing models](./authoring_multibody_simulation.ipynb#Viewing-models) in the tutorial [Authoring a Multibody Simulation](./authoring_multibody_simulation.ipynb) for an introduction to MeshCat.

In [2]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


## Create compliant-hydroelastic bell pepper in SDFormat

*Make sure you have the MeshCat tab opened in your browser; the link is shown immediately above.*

We will create and visualize a compliant-hydroelastic bell pepper with SDFormat string. We will use `ModelVisualizer` to verify the SDFormat description.

We will specify the inertia matrix in the `<inertial>` block. See [mesh_to_model](https://drake.mit.edu/pydrake/pydrake.multibody.mesh_to_model.html) to compute the inertia matrix.

We will create a visual geometry using a triangle surface mesh from an OBJ file and a collision geometry using a tetrahedral volume mesh from a VTK file.

The `<drake:proximity_properties>` block will control hydroelastic contacts. We will set `<drake:hydroelastic_modulus>` to 1e6 Pascals.

In the MeshCat tab, you should toggle the "proximity" checkbox to show the collision geometry, which is the tetrahedral mesh that fits the visual geometry's triangle mesh.  See the section *Viewing models* in [authoring_multibody_simulation.ipynb](./authoring_multibody_simulation.ipynb) for more details.

In [6]:
bell_pepper_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="BellPepper">
    <pose>0 0 0 0 0 0</pose>
    <link name="bell_pepper">
      <inertial>
        <pose>0.0005640521230264642 -0.002833397537457271 0.03815647875628022 0 0 0</pose>
        <mass>0.159</mass>
        <inertia>
          <ixx>0.00010097949016743932</ixx>
          <ixy>-1.1685669139771699e-06</ixy>
          <ixz>-3.502804072036138e-06</ixz>
          <iyy>0.00010464533212142979</iyy>
          <iyz>7.025279142677401e-06</iyz>
          <izz>0.00010606517372681945</izz>
        </inertia>
      </inertial>
      <visual name="visual">
        <geometry>
          <mesh>
            <uri>package://drake_models/veggies/yellow_bell_pepper_no_stem_low.obj</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
      </visual>
      <collision name="collision">
        <geometry>
          <mesh>
            <uri>package://drake_models/veggies/yellow_bell_pepper_no_stem_low.vtk</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1e6</drake:hydroelastic_modulus>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
      </collision>
    </link>
  </model>
</sdf>
"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(bell_pepper_sdf, "sdf")
visualizer.Run(loop_once=True)

## Create compliant-hydroelastic bowl in URDF

We will create and visualize a compliant-hydroelastic bowl with URDF string. We will use `ModelVisualizer` to verify the URDF description.

We will specify the inertia matrix in the `<inertial>` block. It was calculated by [mesh_to_model](https://drake.mit.edu/pydrake/pydrake.multibody.mesh_to_model.html).

We will create a visual geometry using a triangle surface mesh from an OBJ file and a collision geometry using a tetrahedral volume mesh from a VTK file.

In the `<drake:proximity_properties>` block, we will set `<drake:hydroelastic_modulus>` to 1e7 Pascals, so the bowl is stiffer than the bell pepper.

In [4]:
# In interactive mode, use a finer tetrahedral mesh.
# In test mode, use a coarser tetrahedral mesh.
bowl_collision_mesh = \
    "evo_bowl_fine44k.vtk" if "TEST_SRCDIR" not in os.environ else "evo_bowl_coarse3k.vtk"
    
bowl_urdf = f"""<?xml version="1.0"?>
<robot name="Bowl">
  <link name="bowl">
    <inertial>
      <mass value="0.5"/>
      <origin xyz="-6.8577139059790044e-06 -1.7473168538940867e-05 -0.003065665933405878"/>
      <inertia ixx="0.0009715521141455593"
               ixy="-1.8443983471817828e-07"
               ixz="2.4082416205698527e-08"
               iyy="0.0009793965751030533"
               iyz="6.72877272895659e-08"
               izz="0.001592061359500251"/>
    </inertial>
    <visual name="visual">
      <geometry>
        <mesh filename="package://drake_models/dishes/bowls/evo_bowl_no_mtl.obj"/>
      </geometry>
      <material>
       <color rgba="0.9 0.8 0.7 0.5"/>
      </material>
    </visual>
    <collision name="collision">
      <geometry>
        <mesh filename="package://drake_models/dishes/bowls/{bowl_collision_mesh}"/>
      </geometry>
      <drake:proximity_properties>
        <drake:compliant_hydroelastic/>
        <drake:hydroelastic_modulus value="1e7"/>
        <drake:mu_dynamic value="0.5"/>
        <drake:hunt_crossley_dissipation value="1.25"/>
      </drake:proximity_properties>
    </collision>
  </link>
</robot>
"""

# Visualize the URDF string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(bowl_urdf, "urdf")
visualizer.Run(loop_once=True)

RuntimeError: <literal-string>.urdf:24: error: URI 'package://drake_models/dishes/bowls/evo_bowl_fine44k.vtk' resolved to '/home/dan/.cache/drake/package_map/f1b37fb0f82be4b221bac0497d62cc4505afd8ddf9f46062e97f5797b8d73baa-939fc07b168bc8d8bbd6997dd870eff39a4c5d026fec733c8d323e0bef4dd033/dishes/bowls/evo_bowl_fine44k.vtk' which does not exist.

## Create compliant-hydroelastic table top in URDF

The following URDF string specifies a compliant-hydroelastic box for a table top.  We demonstrate how to set relevant hydroelastic properties in URDF; however, Drake prefers SDFormat to URDF.

Both the `<visual>` and `<collision>` geometries are boxes of the same size.

In the `<drake:proximity_properties>` block, we will set `<drake:hydroelastic_modulus>` to 1e7 Pascals.


We do not specify the inertia matrix of the table top because, in the next section when we set up `Diagram`, we will fix the table top to the world frame. It will not move.

In [8]:
table_top_urdf = """<?xml version="1.0"?>
<robot name="TableTop">
  <link name="table_top_link">
    <visual name="visual">
      <geometry>
        <box size="0.6 1.0 0.05"/>
      </geometry>
      <material>
       <color rgba="0.9 0.8 0.7 0.5"/>
      </material>
    </visual>
    <collision name="collision">
      <geometry>
        <box size="0.6 1.0 0.05"/>
      </geometry>
      <drake:proximity_properties>
        <drake:compliant_hydroelastic/>
        <drake:hydroelastic_modulus value="1e7"/>
        <drake:mu_dynamic value="0.5"/>
        <drake:hunt_crossley_dissipation value="1.25"/>
      </drake:proximity_properties>
    </collision>
  </link>
  <frame name="top_surface" link="table_top_link" xyz="0 0 0.025" rpy="0 0 0"/>
</robot>
"""

# Visualize the URDF string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(table_top_urdf, "urdf")
visualizer.Run(loop_once=True)

## Create Diagram of the scene

The function `add_scene()` below will create a scene using the assets that we created. It will use `Parser` to add the URDF and SDFormat strings into the scene. After this step, the next section will add visualization.

In [9]:
def add_scene(time_step):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlant(
        MultibodyPlantConfig(
            time_step=time_step,
            discrete_contact_approximation="lagged"),
        builder)
    parser = Parser(plant)

    # Load the assets that we created.
    parser.AddModelsFromString(table_top_urdf, "urdf")
    parser.AddModelsFromString(bowl_urdf, "urdf")
    parser.AddModelsFromString(bell_pepper_sdf, "sdf")

    # Weld the table top to the world so that it's fixed during simulation.
    # The top surface passes the world's origin.
    plant.WeldFrames(plant.world_frame(), 
                     plant.GetFrameByName("top_surface"))

    # Finalize the plant after loading the scene.
    plant.Finalize()

    # Place the bowl on top of the table.
    X_WB = RigidTransform(p=[0, 0, 0.03])
    plant.SetDefaultFreeBodyPose(plant.GetBodyByName("bowl"), X_WB)
    
    # Drop the bell pepper from above the rim of the bowl. 
    X_WC = RigidTransform(p=[-0.06, 0, 0.30])
    plant.SetDefaultFreeBodyPose(plant.GetBodyByName("bell_pepper"), X_WC)

    return builder, plant

## Set up visualization

The function `add_viz()` below will create visualization. First we will call `ApplyVisualizationConfig()` to visualize our assets. At this step we will set `publish_contacts=False`, so we can customize contact visualization afterwards. 

To visualize contact result, we will add `ContactVisualizer` with `newtons_per_meter= 20` and `newtons_meters_per_meter= 0.1`. It will draw a red arrow of length 1 meter for each force of 20 newtons and a blue arrow of length 1 meter for each torque of 0.1 newton\*meters. The next section will run the simulation.

In [10]:
def add_viz(builder, plant):
    ApplyVisualizationConfig(
        builder=builder, meshcat=meshcat,
        config=VisualizationConfig(
                 publish_contacts=False))    
    ContactVisualizer.AddToBuilder(
        builder=builder, plant=plant, meshcat=meshcat,
        params=ContactVisualizerParams(
                 newtons_per_meter= 20,
                 newton_meters_per_meter= 0.1))

## Run simulation

We will run the simulation. In MeshCat, the red arrow will represent the force `f`, and the blue arrow will represent the torque `tau`. You should see the contact patch moving around together with the force and torque vectors.

After running the code below, playback with `timeScale` = 0.1 to appreciate the contact dynamics. You should see the force and torque vectors oscillate synchronously with the rocking bell pepper and bowl. See the section *Playback recording of the simulation* in [hydroelastic_contact_basics.ipynb](./hydroelastic_contact_basics.ipynb) for more details.

Currently playing back the simulation will show contact force and torque correctly; however, it does not show contact patch appropriately, which could be confusing. Issue [19142](https://github.com/RobotLocomotion/drake/issues/19142) explains the problem in more details.

In [11]:
# Clear MeshCat window from the previous blocks.
meshcat.Delete()
meshcat.DeleteAddedControls()

time_step = 1e-2
builder, plant = add_scene(time_step)
add_viz(builder, plant)

diagram = builder.Build()

simulator = Simulator(diagram)

# In interactive mode, simulate for longer time.
# In test mode, simulate for shorter time.
sim_time = 2 if "TEST_SRCDIR" not in os.environ else 0.01

meshcat.StartRecording()
simulator.set_target_realtime_rate(1)
simulator.AdvanceTo(sim_time)
meshcat.StopRecording()
meshcat.PublishRecording()

RuntimeError: <literal-string>.urdf:24: error: URI 'package://drake_models/dishes/bowls/evo_bowl_fine44k.vtk' resolved to '/home/dan/.cache/drake/package_map/f1b37fb0f82be4b221bac0497d62cc4505afd8ddf9f46062e97f5797b8d73baa-939fc07b168bc8d8bbd6997dd870eff39a4c5d026fec733c8d323e0bef4dd033/dishes/bowls/evo_bowl_fine44k.vtk' which does not exist.

## Download simulation result into a html file for sharing

You can download the simulation result into a self-contained html file, allowing others to playback the simulated results without simulating. The following code prints the URL for downloading. Click on the printed URL to download.

In [ ]:
print(f"{meshcat.web_url()}/download")

## Further reading

* [Hydroelastic Contact User Guide](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)

* Elandt, R., Drumwright, E., Sherman, M., & Ruina, A. (2019, November). A pressure field model for fast, robust approximation of net contact force and moment between nominally rigid objects. In 2019 IEEE/RSJ International Conference on Intelligent Robots and Systems(IROS) (pp. 8238-8245). IEEE. [link](https://arxiv.org/abs/1904.11433)

* Masterjohn, J., Guoy, D., Shepherd, J., & Castro, A. (2022). Velocity Level Approximation of Pressure Field Contact Patches. IEEE Robotics and Automation Letters 7, no. 4 (2022): 11593-11600. [link](https://arxiv.org/abs/2110.04157v2)

* Elandt, R. (2022, December). Pressure Field Contact. Dissertation. Cornell University. [link](https://ecommons.cornell.edu/handle/1813/112919)